<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Project-Tasks" data-toc-modified-id="Project-Tasks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Project Tasks</a></span></li><li><span><a href="#Preamble" data-toc-modified-id="Preamble-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preamble</a></span></li><li><span><a href="#Master-Table" data-toc-modified-id="Master-Table-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Master Table</a></span></li><li><span><a href="#Primary-Pipeline" data-toc-modified-id="Primary-Pipeline-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Primary Pipeline</a></span></li><li><span><a href="#Creating-Master-Bias" data-toc-modified-id="Creating-Master-Bias-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Creating Master Bias</a></span><ul class="toc-item"><li><span><a href="#Bias-Subtract-All-Images" data-toc-modified-id="Bias-Subtract-All-Images-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Bias Subtract All Images</a></span></li></ul></li><li><span><a href="#Creating-Weighted-Dome-Flats-for-Each-Filter" data-toc-modified-id="Creating-Weighted-Dome-Flats-for-Each-Filter-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Creating Weighted Dome Flats for Each Filter</a></span><ul class="toc-item"><li><span><a href="#Dome-Flat-Correct-Every-File" data-toc-modified-id="Dome-Flat-Correct-Every-File-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Dome Flat Correct Every File</a></span></li></ul></li><li><span><a href="#Astrometry" data-toc-modified-id="Astrometry-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Astrometry</a></span><ul class="toc-item"><li><span><a href="#Renaming-*_new.fits-to-something-reasonable" data-toc-modified-id="Renaming-*_new.fits-to-something-reasonable-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Renaming *_new.fits to something reasonable</a></span></li></ul></li><li><span><a href="#Stack-Science-Images" data-toc-modified-id="Stack-Science-Images-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Stack Science Images</a></span><ul class="toc-item"><li><span><a href="#Parse-Filenames-by-Type" data-toc-modified-id="Parse-Filenames-by-Type-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Parse Filenames by Type</a></span></li><li><span><a href="#Stack-by-Filter" data-toc-modified-id="Stack-by-Filter-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Stack by Filter</a></span></li></ul></li><li><span><a href="#Photometry" data-toc-modified-id="Photometry-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Photometry</a></span></li><li><span><a href="#Depreciated" data-toc-modified-id="Depreciated-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Depreciated</a></span><ul class="toc-item"><li><span><a href="#Creating-Sky-Flat-for-Each-Filter" data-toc-modified-id="Creating-Sky-Flat-for-Each-Filter-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Creating Sky Flat for Each Filter</a></span></li><li><span><a href="#Smooth-Sky-Flats" data-toc-modified-id="Smooth-Sky-Flats-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Smooth Sky Flats</a></span></li><li><span><a href="#Sky-Flat-Correct-Science" data-toc-modified-id="Sky-Flat-Correct-Science-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Sky Flat Correct Science</a></span></li></ul></li></ul></div>

# Introduction

ASTR 301 : Data Reduction

Kaimi Kahihikolo

Updated: **2018-03-22**

# Project Tasks

1. <del> Parse Master Table

1. <del> Generate master bias

    2. <del> Print median of master bias

1. (optional) <del> Perform Statistics on overscan region

    2. <del> Print median value in each overscan region
    
    2. Append stats to dataframe
    
1. (optional) Determine amount of "bad pixels" per image. Number of pixels 3$\sigma$ above the median
    
1. <del> Perform Bias Subtraction on flat and science images

    2. Over scan correct
    2. Trim

1. <del> Generate weighted dome flat

    2. <del> R-Filter
    2. <del> I-Filter
    2. <del> Z-Filter
    
1. <del> Apply dome flat to all files

1. <del> Generate Sky Flat REMOVED

    2. <del> R-Filter
    2. <del> I-Filter
    2. <del> Z-Filter
    
1. <del>Smooth sky flat REMOVED

1. <del>Sky Flat Correct science images REMOVED
    
1. Reproject for each field (targets), ie ROSS711
    
    2. CCDPROC WCS project
        
1. Stack science images in each filter
    
1. Photometry
    
    2. Find objects
    2. Measure each object
    

Deep Detection image:

$r + i + z$

# Preamble

In [1]:
import numpy as np
import pandas as pd
import os
from astropy.io import fits
import ccdproc as cp
import matplotlib.pyplot as plt

In [2]:
main_dir = "/Volumes/LaCie/School/ASTR_301/A2/"
data_dir = main_dir+"data/"
out_dir = main_dir+"out/"

# Master Table

In [147]:
df = pd.read_csv(data_dir+'Tek2007Nov05.log', sep=", ", 
                    header=None, names=["Filename", 
                                        "AIRMASS",
                                        "EXPTIME", 
                                        "Filter", 
                                        "Type"])

/Users/kaimibk/miniconda3/envs/UHenv/lib/python3.6/site-packages/ipykernel/__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [148]:
#matchers = ["dome", "sky", "Sky", "bias"]
#sci_list = [df["Filename"][s] for s in range(len(df["Filename"])) if not any(xs in df["Type"][s] for xs in matchers)]

sci_list = df[df.index > 40]

In [47]:
df.head(50)

,Filename,AIRMASS,EXPTIME,Filter,Type
0,tek001.fits,1.000,0.000,NaN,"""bias"""
1,tek002.fits,1.000,0.000,NaN,"""bias"""
2,tek003.fits,1.000,0.000,NaN,"""bias"""
3,tek004.fits,1.000,0.000,NaN,"""bias"""
4,tek005.fits,1.000,0.000,NaN,"""bias"""
5,tek006.fits,1.000,0.000,NaN,"""bias"""
6,tek007.fits,1.000,0.000,NaN,"""bias"""
7,tek008.fits,1.000,0.000,NaN,"""bias"""
8,tek009.fits,1.000,0.000,NaN,"""bias"""
9,tek010.fits,1.000,0.000,NaN,"""bias"""


# Primary Pipeline

In [209]:
class fits_image:
    '''
    Data Processing Pipline for fits images
        Attributes:
            image (str) : File name and extension
            data (CCDData) : Extracted fits image
            header (CCDData) : Parsed Header with keywords  
    '''
    def __init__(self, infile):
        self.image = infile
        self.data = cp.CCDData.read(data_dir+infile, 
                                    unit="adu")
        
        self.header = self.data.header
        
        print("Inspecting %s"%infile)

        
        
    
    def view_image(self, save=True):
        '''
        View the image
        
        args (bool) : Save out .png (default is True) 
        
        returns:
            Pyplot image 
        '''
        plt.clf()
        plt.imshow(self.data, cmap=plt.cm.gray, 
                       vmin=0, vmax=20000)
        
        plt.colorbar()

        plt.grid(color="white", alpha=0.7)
        
        
        if save == True:
            plt.savefig(out_dir+"view_image/"+self.image[0:-8]+".png",
                       bbox_inches = "tight",
                       dpi=240)
        
        elif save == False:
            plt.show()
    
    def os_stats(self, save=False):
        '''
        Conduct stats on overscan region
        
        Args (bool) : Save out stats to dataframe (default is False) 
        
        returns:
            (float) overs scan region Median
            (float) Mean
            (float) Standard Deviation
        '''
        os_region = self.data[2049:2080, 1:2068]
        
        if save == True:
            df.append(pd.DataFrame({'os_median': np.median(os_region), 
                                    'os_mean': np.mean(os_region),
                                   'os_std' : np.std(os_region)}, 
                                   index=[0]), 
                              ignore_index=True)
        
        else:
            print("\t Median : %s"+
                 "\n\t Average : %s"+
                 "\n\t $\sigma$ : %s"%(np.median(os_region),
                                        np.mean(os_region),
                                        np.std(os_region)))

    def bias_sub(self, bias_file):
        '''
        Conduct Bias Subtraction on images and saves as *_b.fits
        Args (CCDData): Master Bias File
        
        Returns:
            Saves fits image to "bias_sub/" directory
        
        Raises:
            IOError: Bias subtracted file already exists
            ValueError: File is wrong shape (pre-corrected)
                trims file and saves out as *_b.fits
            KeyError: "BIASSEC" key doesn't exist
        '''
        
        try:
            bias_subtracted = cp.subtract_bias(self.data, 
                                           master_bias)

            os_subtracted = cp.subtract_overscan(bias_subtracted, 
                    fits_section=self.header["BIASSEC"],
                    overscan_axis=(0,1),
                    median=True)

            trimmed = cp.trim_image(os_subtracted[:2048, :2048])
            

            cp.fits_ccddata_writer(trimmed, 
                       out_dir+"bias_sub/"+self.image[0:-8]+"_b.fits")
        
        except IOError:
            print("\t ...File already exists...")
        
        except ValueError:
            trimmed = cp.trim_image(self.data[:2048, :2048])
            
            cp.fits_ccddata_writer(trimmed, 
                       out_dir+"bias_sub/"+self.image[0:-8]+"_b.fits")
            
            print("\t ...Skipping...wrong shape...")
            
        except KeyError:
            print("\t ...Skipping...missing Key...")

        
    
    def dome_flat_corr(self):
        '''
        Dome  flat correct images and saves as *fb.fits
        
        Returns:
            Saves fits image to "flat_corr/" directory
            
        Raises:
            IOError: Flat corrected file already exists
            ValueError: File is wrong shape
        '''
        try:
            image = out_dir+"bias_sub/"+self.image[0:6]+'_b.fits'

            image_data = cp.fits_ccddata_reader(image,
                                               unit="adu") 


            master_flat_file = out_dir+"flats/dome_flat_"+self.header["FILTER"][0]+".fits"

            master_flat = cp.fits_ccddata_reader(master_flat_file,
                                                unit="adu")

            flat_corr_image = cp.flat_correct(image_data, master_flat)

            cp.fits_ccddata_writer(flat_corr_image,
                          out_dir+"flat_corr/"+self.image[0:-8]+"_f.fits")

        except ValueError:
            print("\t ...Skipping...")
            
        except IOError:
            print("\t ...File already exists...")
    
    def sky_flat_corr(self):
        '''
        Dome  flat correct images and saves as *fb.fits
        
        Returns:
            Saves fits image to "flat_corr/" directory
            
        Raises:
            IOError: Flat corrected file already exists
            ValueError: File is wrong shape
        '''
        try:
            image = out_dir+"flat_corr/"+self.image[0:6]+'_f.fits'

            image_data = cp.fits_ccddata_reader(image,
                                               unit="adu") 


            master_flat_file = out_dir+"median_filter/sky_flat_"+self.header["FILTER"][0]+"_s.fits"

            master_flat = cp.fits_ccddata_reader(master_flat_file,
                                                unit="adu")

            flat_corr_image = cp.flat_correct(image_data, master_flat)

            cp.fits_ccddata_writer(flat_corr_image,
                          out_dir+"sky_flat_corr/"+self.image[0:-8]+"_bfs.fits")

        except ValueError:
            print("\t ...Skipping...")
            

    def transform(self):
        '''
        Perform astrometry on the image using astrolib solve_field

        raises "Correction Exists...skipping..." if photometry already performed
        '''
        if os.path.isfile(out_dir+"astrometry/"+self.image[0:6]+"_f_new.fits") == True:
            print("\t Correction Exists...skipping...")

        else:
            try:
                ac.solve_field(out_dir+"flat_corr/"+self.image[0:6]+"_f.fits")

                move(out_dir+"flat_corr/"+self.image[0:6]+"_f_new.fits", 
                     out_dir+"astrometry/"+self.image[0:6]+"_f_new.fits")     
                
            except FileNotFoundError:
                print("Could not be solved...skipping")
                
            
            
    def reprojection(self, reference):
        if os.path.isfile(out_dir+"test/"+self.image[0:6]+".fits") == True:
            print("\t Reprojection Exists...skipping...")
        
        try:
            hdu2 = fits.open(out_dir+"astrometry/"+self.image)[0]

            array, footprint = reproject_interp(hdu2, reference.header)
            fits.writeto(out_dir+"test/"+self.image[0:6]+".fits", array, 
                         reference.header, overwrite=True)
        
        except FileNotFoundError:
            print("File missing...skipping...")


# Creating Master Bias

In [6]:
bias_list = (df["Filename"][df["EXPTIME"] == 0])

In [8]:
def gen_master_bias(bias_list, over_scan=False):
    if (os.path.exists(out_dir+"master_bias.fits")):
        print("\t ...Master Bias already exists...")

    else:
        bias_frames = []

        for image in bias_list:
            print("Inspecting Image ... %s")%(image)

            frame = cp.fits_ccddata_reader(data_dir+image+".gz", 
                                           hdu=0, 
                                           unit="adu")

            if over_scan == True:
                frame = cp.subtract_overscan(frame, 
                            fits_section=frame.header["BIASSEC"],
                            overscan_axis=(0,1),
                            median=True)



            bias_frames.append(frame)
        master_bias = cp.combine(bias_frames,method="median")
        cp.fits_ccddata_writer(frame, out_dir+"master_bias.fits")

In [8]:
gen_master_bias(bias_list)

	 ...Master Bias already exists...


<a id="bias-sub"></a>
## Bias Subtract All Images

In [23]:
master_bias = cp.CCDData.read(out_dir+"master_bias.fits",
                        unit="adu")

k = 0

for filename in os.listdir(data_dir):
    if filename.endswith(".fits.gz") and k > 9:
        my_image = fits_image(filename)
        
        my_image.bias_sub(master_bias)
    else:
        k += 1
        

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
Inspecting tek010.fits.gz
Inspecting tek011.fits.gz
	 ...Skipping...wrong shape...
Inspecting tek012.fits.gz
	 ...Skipping...wrong shape...
Inspecting tek013.fits.gz
Inspecting tek014.fits.gz
Inspecting tek015.fits.gz
Inspecting tek016.fits.gz
Inspecting tek017.fits.gz
Inspecting tek018.fits.gz
Inspecting tek019.fits.gz
Inspecting tek020.fits.gz
Inspecting tek021.fits.gz
Inspecting tek022.fits.gz
Inspecting tek023.fits.gz
Inspecting tek024.fits.gz
Inspecting tek025.fits.gz
Inspecting tek026.fits.gz
Inspecting tek027.fits.gz
Inspecting tek028.fits.gz
Inspecting tek029.fits.gz
Inspecting tek030.fits.gz
Inspecting tek031.fits.gz
Inspecting tek032.fits.gz
Inspecting tek033.fits.gz
Inspecting tek034.fits.gz
Inspecting tek035.fits.gz
Inspecting tek036.fits.gz
Inspecting tek037.fits.gz
Inspecting tek038.fits.gz
Inspecting tek039.fits.gz
Inspecting tek040.fits.gz
Inspecting tek041.fit

# Creating Weighted Dome Flats for Each Filter

In [27]:
dome_list = []
for i in range(len(df["Type"])):
    if "dome" in df["Type"][i]:
        dome_list.append((df["Filename"][i],df["Type"][i][1:-1]))

In [43]:
def mean_image_stack(image_list, out_name):
    
    if (os.path.exists(out_dir+"flats/"+out_name)):
        print "\t ...Master flat already exists..."
    
    else:
        frames = []
        weights = []
        for image in image_list:
            frame = cp.CCDData.read(out_dir+"bias_sub/"+image[0][0:6]+"_b.fits", 
                                           unit="adu")
            frames.append(frame)
            median_frame = np.median(frame)
            weights.append(median_frame)
        
        for i in range(len(frames)):
            frames[i].data = frames[i].data * (weights[i]/max(weights))
        
        stack = cp.combine(frames, method="average")
        cp.fits_ccddata_writer(stack, out_dir+"flats/"+out_name)

In [44]:
mean_image_stack(dome_list[0:5], "dome_flat_r.fits")
mean_image_stack(dome_list[5:10], "dome_flat_i.fits")
mean_image_stack(dome_list[10:14], "dome_flat_z.fits")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
I

## Dome Flat Correct Every File

In [45]:
k = 0
for filename in os.listdir(data_dir):
    if filename.endswith(".fits.gz") and k>10:
        my_image = fits_image(filename)

        my_image.dome_flat_corr()
    
    else:
        k +=1

Inspecting tek011.fits.gz
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
Inspecting tek012.fits.gz
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
Inspecting tek013.fits.gz
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
Inspecting tek014.fits.gz
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [ccdproc.ccddata]
Inspecti

# Astrometry

In [49]:
from astrolib import astronomy
from shutil import move
            
ac = astronomy.AstCalc()

for image in sci_list["Filename"]:
    my_image = fits_image(image)
    my_image.transform()

Inspecting tek046.fits
	 Correction Exists...skipping...
Inspecting tek047.fits
	 Correction Exists...skipping...
Inspecting tek048.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek048_f.fits cannot be solved!
Could not be solved...skipping
Inspecting tek049.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek049_f.fits cannot be solved!
Could not be solved...skipping
Inspecting tek050.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek050_f.fits --> /Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek050_f_new.fits: solved!
Inspecting tek051.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek051_f.fits --> /Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek051_f_new.fits: solved!
Inspecting tek052.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek052_f.fits --> /Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek052_f_new.fits: solved!
Inspecting tek053.fits
/Volumes/LaCie/School/ASTR_301/A2/out/flat_corr/tek053_f.fits --> /Volumes/LaCie/School/ASTR_301/A2/out

## Renaming *_new.fits to something reasonable

In [55]:
for filename in os.listdir(out_dir+"astrometry/"):
    if filename.endswith(".fits"):
        os.rename(out_dir+"astrometry/"+filename, out_dir+"astrometry/"+filename[0:6]+".fits")

# Stack Science Images

https://reproject.readthedocs.io/stable/

After all the images have astrometry, you can use a set of coordinates like,

```python
from astropy.io import fits
from astropy.wcs import WCS

hdu1 = fits.open("reference_image.fits")[0]
hdu2 = fits.open("image_to_reproject.fits")[0]

from reproject import reproject_interp
array, footprint = reproject_interp(hdu2, hdu1.header)
fits.writeto("reprojected_image.fits", array, hdu1.header, overwrite=True)
```

In [149]:
from reproject import reproject_interp

objects = np.sort(sci_list["Type"].unique())

## Parse Filenames by Type

In [195]:
k = 0

master_dict = {}

sci_list = sci_list.sort_values(by=["Type"])

obj_files = []

for idx in range(len(sci_list)):
    if sci_list["Type"].values[idx] == objects[k]:
        #print(sci_list["Filename"].values[idx])
        obj_files.append(np.array([sci_list["Filename"].values[idx], 
                         sci_list["Filter"].values[idx]]))
    
    else:
        obj_files = []
        k += 1
        
        if sci_list["Type"].values[idx] == objects[k]:
            #print(sci_list["Filename"].values[idx])
            obj_files.append([sci_list["Filename"].values[idx],
                             sci_list["Filter"].values[idx]])
        
    master_dict.update({objects[k] : obj_files})

In [1]:
master_dict.keys()
#np.rot90(master_dict['2M2234'])[1][0]

NameError: name 'master_dict' is not defined

In [212]:
for i in master_dict.keys():
    print("\t Inspecting Object {}".format(i))
    try:
        reference = fits.open(out_dir+"astrometry/"+np.rot90(master_dict[i])[1][0])[0]
    
    except FileNotFoundError:
        reference = fits.open(out_dir+"astrometry/"+np.rot90(master_dict[i])[1][1])[0]
        
    for filename in np.rot90(master_dict[i])[1]:
        my_image = fits_image(filename)
        my_image.reprojection(reference)
        

	 Inspecting Object 2M2234
Inspecting tek139.fits
	 Reprojection Exists...skipping...
Inspecting tek141.fits
	 Reprojection Exists...skipping...
Inspecting tek142.fits
	 Reprojection Exists...skipping...
Inspecting tek138.fits
	 Reprojection Exists...skipping...
Inspecting tek137.fits
	 Reprojection Exists...skipping...
Inspecting tek136.fits
	 Reprojection Exists...skipping...
Inspecting tek135.fits
	 Reprojection Exists...skipping...
Inspecting tek140.fits
	 Reprojection Exists...skipping...
	 Inspecting Object 2M2249
Inspecting tek128.fits
	 Reprojection Exists...skipping...
Inspecting tek114.fits
	 Reprojection Exists...skipping...
Inspecting tek122.fits
	 Reprojection Exists...skipping...
Inspecting tek116.fits
	 Reprojection Exists...skipping...
Inspecting tek123.fits
	 Reprojection Exists...skipping...
Inspecting tek118.fits
	 Reprojection Exists...skipping...
Inspecting tek115.fits
	 Reprojection Exists...skipping...
Inspecting tek124.fits
	 Reprojection Exists...skipping...
In

In [208]:
master_dict["SA105_815"]

[['tek054.fits', 'z'],
 array(['tek053.fits', 'i'], dtype='<U11'),
 array(['tek055.fits', 'z'], dtype='<U11'),
 array(['tek056.fits', 'r'], dtype='<U11'),
 array(['tek057.fits', 'r'], dtype='<U11'),
 array(['tek052.fits', 'i'], dtype='<U11')]

## Stack by Filter

In [224]:
for i in master_dict.keys():
    r_list = []
    i_list = []
    z_list = []
    
    for idx in range(len(np.rot90(master_dict[i])[1])):
        if np.rot90(master_dict[i])[0][idx] == "r":
            r_list.append(np.rot90(master_dict[i])[1][idx])
        elif np.rot90(master_dict[i])[0][idx] == "i":
            i_list.append(np.rot90(master_dict[i])[1][idx])
        elif np.rot90(master_dict[i])[0][idx] == "z":
            z_list.append(np.rot90(master_dict[i])[1][idx])
    
    try:
        stack = cp.combine([cp.CCDData.read(out_dir+"test/"+j[0:6]+".fits", 
                                                   unit="adu") for j in r_list], 
                               method="average")

        cp.fits_ccddata_writer(stack, out_dir+"final/"+i+"_r.fits")
    
    except IndexError:
        print("Skipping")
    except FileNotFoundError:
        print("Skipping")
    
    try:
        stack = cp.combine([cp.CCDData.read(out_dir+"test/"+j[0:6]+".fits", 
                                               unit="adu") for j in i_list], 
                           method="average")

        cp.fits_ccddata_writer(stack, out_dir+"final/"+i+"_i.fits")
    
    except IndexError:
        print("Skipping")
    except FileNotFoundError:
        print("Skipping")
    
    try:  
        stack = cp.combine([cp.CCDData.read(out_dir+"test/"+j[0:6]+".fits", 
                                           unit="adu") for j in z_list], 
                       method="average")
    
        cp.fits_ccddata_writer(stack, out_dir+"final/"+i+"_z.fits")
            
    except IndexError:
        print("Skipping")
    except FileNotFoundError:
        print("Skipping")

Skipping
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Skipping
Skipping
INFO: using the unit adu pass

In [220]:
np.rot90(master_dict[i])[0]
print(i)

2M2234


In [226]:
master_dict["2M2249"]

[['tek128.fits', 'z'],
 array(['tek114.fits', 'z'], dtype='<U11'),
 array(['tek122.fits', 'z'], dtype='<U11'),
 array(['tek116.fits', 'z'], dtype='<U11'),
 array(['tek123.fits', 'z'], dtype='<U11'),
 array(['tek118.fits', 'z'], dtype='<U11'),
 array(['tek115.fits', 'z'], dtype='<U11'),
 array(['tek124.fits', 'z'], dtype='<U11'),
 array(['tek127.fits', 'z'], dtype='<U11'),
 array(['tek117.fits', 'z'], dtype='<U11'),
 array(['tek145.fits', 'z'], dtype='<U11'),
 array(['tek130.fits', 'z'], dtype='<U11'),
 array(['tek126.fits', 'z'], dtype='<U11'),
 array(['tek120.fits', 'z'], dtype='<U11'),
 array(['tek129.fits', 'z'], dtype='<U11'),
 array(['tek146.fits', 'z'], dtype='<U11'),
 array(['tek143.fits', 'z'], dtype='<U11'),
 array(['tek144.fits', 'z'], dtype='<U11'),
 array(['tek147.fits', 'z'], dtype='<U11'),
 array(['tek121.fits', 'z'], dtype='<U11')]

# Photometry

In [201]:
df["Filter"].unique()

array([nan, 'r', 'i', 'z'], dtype=object)

# Depreciated

## Creating Sky Flat for Each Filter

sky_list = []
for i in range(len(df["Type"])):
    if "sky" in df["Type"][i] or "Sky" in df["Type"][i]:
        sky_list.append((df["Filename"][i],df["Type"][i][1:-1]))

def mean_image_stack(image_list, out_name):
    
    if (os.path.exists(out_dir+"flat_corr/"+out_name)):
        print "\t ...Master flat already exists..."
    
    else:
        frames = []
        weights = []
        for image in image_list:
            frame = cp.CCDData.read(out_dir+"bias_sub/"+image[0][0:6]+"_b.fits", 
                                           unit="adu")
            frames.append(frame)
            median_frame = np.median(frame)
            weights.append(median_frame)
        
        for i in range(len(frames)):
            frames[i].data = frames[i].data * (weights[i]/max(weights))
        
        stack = cp.combine(frames, method="average")
        cp.fits_ccddata_writer(stack, out_dir+"flats/"+out_name)

mean_image_stack(sky_list[7:12], "sky_flat_r.fits")
mean_image_stack(sky_list[12:17], "sky_flat_i.fits")
mean_image_stack(sky_list[0:7], "sky_flat_z.fits")

## Smooth Sky Flats

def median_filter(filename):
    data = cp.CCDData.read(out_dir+"flats/"+filename)
    filt = cp.median_filter(data, 5)
    cp.fits_ccddata_writer(filt,
              out_dir+"median_filter/"+filename[0:-5]+"_s.fits")


for filename in os.listdir(out_dir+"flats/"):
    if filename.startswith("sky_flat_"):
        my_image = median_filter(filename)

## Sky Flat Correct Science

In [9]:
sci_list = df["Filename"][41:]

for filename in sci_list:
    my_image = fits_image(filename+".gz")
    my_image.sky_flat_corr()